# STT sample

## Ses Kaydını Metne Çeviren Uygulama Örneği

Model: [openai/whisper-base](https://huggingface.co/openai/whisper-base)

Dataset: [Nexdata/English_Emotional_Speech_Data_by_Microphone](https://huggingface.co/datasets/Nexdata/English_Emotional_Speech_Data_by_Microphone/viewer/default/train?views%5B%5D=train&row=0)

In [1]:
import torch
print("CUDA durumu:", torch.cuda.is_available())
print("Kullanılan cihaz:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


CUDA durumu: True
Kullanılan cihaz: cuda


In [2]:
!pip install openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 41.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=128a2857166c885eed25c4aa1666192197a37ff414ef29b1446dacae0e681593
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [4]:
import whisper

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base", device=device)

result = model.transcribe("/content/audio.wav")
print(result["text"])


 After the accident, Bill's sister is finally feeling better.


## Mikrofonla Canlı Konuşmayı “Yaklaşık Gerçek Zamanlı” Metne Çeviren Uygulama Örneği

Google Colab (T4 GPU) üzerinde çalışacak, mikrofondan kısa parçalar (chunk) kaydeden ve bu parçaları “yaklaşık gerçek zamanlı” olarak metne çeviren bir örnek notebook kodu veriyorum. Adım adım çalıştır: her hücreyi sırayla çalıştır, sonra Start düğmesine bas ve konuş — her 2–4 saniyelik parça Colab’a gönderilip model tarafından anında transkribe edilecek ve birikimli transcript ekranda görünecek.

Notlar / öneriler

Colab’da GPU (T4) seçili olduğundan emin ol: Runtime → Change runtime type → Hardware accelerator: GPU.

Daha düşük gecikme için kısa CHUNK_SECONDS (ör. 2–3) kullan. Model boyutu ile gecikme/kalite arasında takas var: tiny/base hızlı, small orta, medium/large daha doğru ama yavaş ve GPU belleği gerektirir.

Bu örnek openai-whisper kullanır (local Whisper). Colab kurulumu biraz zaman alabilir.

In [17]:
# !! Çalıştırmadan önce runtime GPU seçili olsun !!
# Kurulum (ffmpeg, whisper, soundfile)
!apt-get update -qq && apt-get install -y -qq ffmpeg
!pip install -q -U openai-whisper soundfile

# (isteğe bağlı) faster-whisper veya başka optimizasyonlar isterseniz buraya ekleyin.


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [18]:
import whisper
import torch
from IPython.display import display, HTML, clear_output
from google.colab import output
import base64
import io
import soundfile as sf
import os

# Ayarlar
MODEL_NAME = "small"   # "tiny", "base", "small", "medium", "large" (bellek ve hız gereksinimi farklı)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CHUNK_SECONDS = 3      # her kayıt parçası kaç saniye olacak (2-4 iyi)
LANG = None            # otomatik tanıma: None, belirli dil için "tr" gibi

print("Cihaz:", DEVICE, " | Model:", MODEL_NAME)

# Modeli yükle (ilk yükleme biraz zaman alır)
model = whisper.load_model(MODEL_NAME, device=DEVICE)

# transcript biriktirme
TRANSCRIPT = []


Cihaz: cuda  | Model: small


100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 60.2MiB/s]


In [30]:
from IPython.display import Javascript
from google.colab import output
import base64

RECORD_SECONDS = 5  # kaç saniye

def record(sec=5, filename="audio.wav"):
    display(Javascript(f"""
    async function record(sec) {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const recorder = new MediaRecorder(stream);
      let data = [];
      recorder.ondataavailable = e => data.push(e.data);
      recorder.start();

      await new Promise(r => setTimeout(r, sec*1000));
      recorder.stop();

      await new Promise(r => recorder.onstop = r);
      const blob = new Blob(data);
      const arrayBuffer = await blob.arrayBuffer();
      const base64String = btoa(
          new Uint8Array(arrayBuffer)
            .reduce((data, byte) => data + String.fromCharCode(byte), '')
      );
      google.colab.kernel.invokeFunction('notebook.save_audio', [base64String], {{}});
    }}
    record({sec});
    """))

    # bu fonksiyon js içinden çağrılacak
    def save_audio(b64):
        audio_bytes = base64.b64decode(b64)
        with open(filename, "wb") as f:
            f.write(audio_bytes)
        print(f"Kayıt tamamlandı: {filename}")
    output.register_callback('notebook.save_audio', save_audio)


In [34]:
record(10, "kayit.wav")  # 5 saniye kayıt yapar


<IPython.core.display.Javascript object>

Kayıt tamamlandı: kayit.wav


In [35]:
import whisper

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base", device=device)

result = model.transcribe("/content/kayit.wav")
print(result["text"])


 How is it going?
